In [ ]:
import os
import sys

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import h5py

sys.path.append(r"D:\codes\python\courtship\utils")
from utils import *
from sleap_utils import *

%load_ext autoreload
%autoreload 2

In [ ]:
h5_dir = r"D:\Fight\rotation\Wei Zhang\projects\courtship song\data\fromZJC\sleap"
h5_fname = "labels.v000.analysis.h5"

h5_path = os.path.join(h5_dir, h5_fname)
fps = 25

In [ ]:
with h5py.File(h5_path, 'r') as f:
    instance_scores = f['instance_scores'][:]
    node_names = f['node_names'][:]
    point_scores = f['point_scores'][:]
    tracking_scores = f['tracking_scores'][:]
    occupancy = f['track_occupancy'][:].transpose()
    tracks_mat = np.transpose(f['tracks'][:], axes=[0,2,1,3])

In [ ]:
track_dfs = readTracks(h5_path)

track_dfs = [addMetrics(track_df, max_displacement=60) for track_df in track_dfs]

temp = [get_flyclips(track_df, min_consecutive=60) for track_df in track_dfs]
flyclips = []
for clip in temp:
    for c in clip:
        flyclips.append(c)

In [ ]:
pairing = get_pairing_info(h5_path)

paired_track_dfs = [pair_tracks(track_dfs=[track_dfs[i] for i in pair]) for pair in pairing]

temp = [get_flyclips(paired_track_df, min_consecutive=60) for paired_track_df in paired_track_dfs]
flyclips_paired = []
for clip in temp:
    for c in clip:
        flyclips_paired.append(c)

In [ ]:
# add timestamps

ts_path = r"D:\Fight\rotation\Wei Zhang\projects\courtship song\data\fromZJC\dvSave-2021_12_29_16_10_58_timestamps.txt"

ts = np.loadtxt(ts_path).astype(np.int64)
ts_dict = dict(zip(list(range(len(ts))), ts))

for i in range(len(flyclips_paired)):
    flyclips_paired[i]['timestamp'] = [ts_dict[frame] for frame in flyclips_paired[i]['frame']]

for i in range(len(flyclips)):
    flyclips[i]['timestamp'] = [ts_dict[frame] for frame in flyclips[i]['frame']]


In [ ]:
# save the clips

save_dir = r"D:\Fight\rotation\Wei Zhang\projects\courtship song\data\out\sleap_out"

task_dir = os.path.join(save_dir, "20220405_0")
if not os.path.exists(task_dir):
    os.mkdir(task_dir)

clips_dir = os.path.join(task_dir, "flyclips")
if not os.path.exists(clips_dir):
    os.mkdir(clips_dir)
for i, clip in enumerate(flyclips):
    clip.to_csv(os.path.join(clips_dir, str(i) + '.txt'))

paired_clips_dir = os.path.join(task_dir, "flyclips_paired")
if not os.path.exists(paired_clips_dir):
    os.mkdir(paired_clips_dir)
for i, clip in enumerate(flyclips_paired):
    clip.to_csv(os.path.join(paired_clips_dir, str(i) + '.txt'))